In [1]:
import pandas as pd
import zipfile
import os
import inspect
import sys
from datetime import datetime

%cd ..

/home/diego/Desktop/monografia


In [2]:
df = pd.read_csv("raw_data/suicidios/suicidios.csv")
todos = pd.read_csv("raw_data/suicidios/todos.csv")

In [3]:
df['DTOBITO'] = pd.to_datetime(df['DTOBITO'])
todos['DTOBITO'] = pd.to_datetime(todos['DTOBITO'])

In [4]:
df['mes_ano'] = df['DTOBITO'].dt.strftime('%Y-%m')
todos['mes_ano'] = todos['DTOBITO'].dt.strftime('%Y-%m')

In [5]:
df_grouped = df.groupby(['mes_ano', 'CODMUNRES']).size().reset_index()
df_grouped.columns = ['date','municipio_ibge', 'suicidios']
todos_grouped = todos.groupby(['mes_ano', 'CODMUNRES']).size().reset_index()
todos_grouped.columns = ['date','municipio_ibge', 'obitos_totais']

In [6]:
df_grouped['date'] = pd.to_datetime(df_grouped['date'])
todos_grouped['date'] = pd.to_datetime(todos_grouped['date'])

In [8]:
URL = 'http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv'
municipios = pd.read_csv(URL, error_bad_lines=False, sep=';', encoding='latin-1', usecols=['IBGE', 'IBGE7']).set_index('IBGE').to_dict()
todos_grouped['municipio_ibge'] = todos_grouped['municipio_ibge'].astype('int32').replace(municipios['IBGE7'])

/tmp/ipykernel_3484059/3520404934.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  municipios = pd.read_csv(URL, error_bad_lines=False, sep=';', encoding='latin-1', usecols=['IBGE', 'IBGE7']).set_index('IBGE').to_dict()


In [17]:
mortes = pd.merge(df_grouped, todos_grouped, on=['date', 'municipio_ibge'], how='left')

In [18]:
mortes['taxa_suicidio'] = mortes['suicidios'] / mortes['obitos_totais']

In [21]:
mortes.to_csv('parsed_data/suicidios/suicidios.csv', index=False)